In [3]:
from lively import Solver, Translation, Rotation,Transform, OrientationLivelinessObjective, OrientationMatchObjective, SmoothnessMacroObjective, PositionMatchObjective, JointMatchObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Constructed CollisionAvoidance Objective ,BoxShape and configured Collision Setting
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=1.0, y =2.0, z=3.0),
rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/pepper.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()
#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objective (smoothness macro)
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=25,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=0.5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5),
      "torso Position": PositionMatchObjective(name="MyPositionMatchObjective", weight=10, link="torso"),
      "positionMatch": PositionMatchObjective(name="R Hand Position", weight=10, link="r_gripper"),
      "r hand orientation": OrientationMatchObjective(name="R Hand Orientation", weight=10, link="r_gripper"),
      "headOrientation": OrientationMatchObjective(name="Gaze",link="Head",weight=7),
      "idleGaze": OrientationLivelinessObjective(name="Idle Gaze", link= "Head", weight=20, frequency=10),
      "LShoulderPitch": JointMatchObjective(name="LShoulderPitch", joint="LShoulderPitch", weight=10),
      "LShoulderRoll": JointMatchObjective(name="LShoulderRoll", joint="LShoulderRoll", weight=10),
      "LElbowYaw": JointMatchObjective(name="LElbowYaw", joint="LElbowYaw", weight=10),
      "LElbowRoll": JointMatchObjective(name="LElbowRoll", joint="LElbowRoll", weight=10),
      "LWristYaw": JointMatchObjective(name="LWristYaw", joint="LWristYaw", weight=10),
      "LHand": JointMatchObjective(name="LHand", joint="LHand", weight=10),
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {"MyGoal1": Translation(x=1.0, y =2.0, z=3.0)},weights = {},time = 0.0)
# Log the initial state
print(state.origin.as_dicts())
print(state.joints)
print(state.proximity)

({'x': 0.0, 'z': 0.0, 'y': 0.0}, {'z': 0.0, 'y': 0.0, 'x': 0.0, 'w': 1.0})
{'LFinger21': 0.30671098947905234, 'RFinger31': 0.3051498806161248, 'LFinger31': 0.30671098947905234, 'LShoulderRoll': 0.7889923684357429, 'LFinger42': 0.30671098947905234, 'RHand': 0.34967585570193005, 'RFinger42': 0.3051498806161248, 'HeadYaw': 0.0039061453328515755, 'RShoulderRoll': -0.7813685268451939, 'LShoulderPitch': 0.0028484825443705563, 'LFinger41': 0.30671098947905234, 'LFinger43': 0.30671098947905234, 'RFinger33': 0.3051498806161248, 'LFinger13': 0.30671098947905234, 'LFinger23': 0.30671098947905234, 'LElbowRoll': -0.7811919854287536, 'LFinger12': 0.30671098947905234, 'LThumb2': 0.30671098947905234, 'RFinger22': 0.3051498806161248, 'RThumb1': 0.3051498806161248, 'RFinger13': 0.3051498806161248, 'RThumb2': 0.3051498806161248, 'RElbowYaw': 7.177040926903798e-05, 'RWristYaw': 0.0004616272661463161, 'RFinger11': 0.3051498806161248, 'LFinger32': 0.30671098947905234, 'RFinger43': 0.3051498806161248, 'LHand